In [6]:
import os
import urllib
from bs4 import BeautifulSoup
import re
import pandas as pd

In [67]:
class Case:
    def __init__(self, year, month, name, url):
        self.year = year
        self.month = month
        self.name = name
        self.url = url
    
    def get_full_url(self):
        return "https://en.wikipedia.org%s" % self.url

In [44]:
MONTHS = {'January'  : 1, 
          'February' : 2, 
          'March'    : 3, 
          'April'    : 4, 
          'May'      : 5,
          'June'     : 6, 
          'July'     : 7, 
          'August'   : 8, 
          'September': 9, 
          'October'  : 10, 
          'November' : 11, 
          'December': 12}

def get_name_and_url(case):
    name = [s for s in case.strings][0]
    url = [link.get('href') for link in case.find_all('a')][0]
    full_url = "https://en/wikipedia.org%s" % url
    return (name,url)

def get_year_and_cases(soup):
    #this works under the assumption that the year is under an <h1>
    for h in soup.find_all('h1'):
        strings = [s for s in h.strings]
        if re.search('^\d\d\d\d$', strings[0]) is not None:
            year = int(strings[0])
            nums = re.findall('\d+\ ', h.next_sibling.next_sibling.string)
            cases = sum([int(n) for n in nums])
    return (year, cases)

def get_cases_for_year(soup):
    #months = {}
    cases = []
    totalcases = 0
    year = get_year_and_cases(soup)[0]
    for h in soup.find_all('h2'):
        strings = [s for s in h.strings]
        if strings[0] in MONTHS.keys():
            month = str(strings[0])
            if h.next_sibling.next_sibling.name == 'ul':
                #cases = []
                for child in h.next_sibling.next_sibling.children:
                    try: 
                        if len(child.contents)> 1:
                            #print("foundCase!")
                            caseinfo = get_name_and_url(child)
                            case = Case(year, MONTHS[month], caseinfo[0], caseinfo[1])
                            #print("\t\t%s" % case.get_full_url())
                            cases.append(case)
                            #print(len(cases))
                            totalcases += 1
                        #else:
                            #print("passed on %s" % [s for s in child.strings][0])
                    except AttributeError:
                        pass
            #months[MONTHS[month]] = cases
    #return months
    #print("\tfound %d cases"%totalcases)
    #print("\tor maybe %d cases"%len(cases))
    return cases

In [47]:
years = [y for y in range(2004,2019)]
indexurl = 'https://en.wikipedia.org/wiki/Wikipedia:Arbitration/Index/Cases/%d'
allCases = []
total = 0
for year in years:
    soup = BeautifulSoup(urllib.request.urlopen(indexurl % year).read(), 'html.parser')
    out = get_year_and_cases(soup)
    total += int(out[1])
    print("%s had %s cases" % out)
    cases = get_cases_for_year(soup)
    print('\tfound %d cases' % len(cases))
    allCases += cases
    

2004 had 35 cases
	found 35 cases
2005 had 100 cases
	found 100 cases
2006 had 116 cases
	found 88 cases
2007 had 97 cases
	found 41 cases
2008 had 41 cases
	found 11 cases
2009 had 30 cases
	found 14 cases
2010 had 12 cases
	found 6 cases
2011 had 16 cases
	found 10 cases
2012 had 11 cases
	found 0 cases
2013 had 12 cases
	found 6 cases
2014 had 11 cases
	found 6 cases
2015 had 18 cases
	found 13 cases
2016 had 5 cases
	found 4 cases
2017 had 4 cases
	found 3 cases
2018 had 6 cases
	found 4 cases


In [48]:
total, len(allCases)

(514, 341)

In [49]:
[case for case in allCases if case.year == 2012]

[]

In [61]:
s12 = BeautifulSoup(urllib.request.urlopen(indexurl % 2012).read(), 'html.parser')
s4 = BeautifulSoup(urllib.request.urlopen(indexurl % 2004).read(), 'html.parser')

In [ ]:
def get_cases_for_year(soup):
    #months = {}
    cases = []
    totalcases = 0
    year = get_year_and_cases(soup)[0]
    for h in soup.find_all('h2'):
        strings = [s for s in h.strings]
        if strings[0] in MONTHS.keys():
            month = str(strings[0])
            if h.next_sibling.next_sibling.name == 'ul':
                #cases = []
                for child in h.next_sibling.next_sibling.children:
                    try: 
                        if len(child.contents)> 1:
                            #print("foundCase!")
                            caseinfo = get_name_and_url(child)
                            case = Case(year, MONTHS[month], caseinfo[0], caseinfo[1])
                            #print("\t\t%s" % case.get_full_url())
                            cases.append(case)
                            #print(len(cases))
                            totalcases += 1
                        #else:
                            #print("passed on %s" % [s for s in child.strings][0])
                    except AttributeError:
                        pass
            #months[MONTHS[month]] = cases
    #return months
    #print("\tfound %d cases"%totalcases)
    #print("\tor maybe %d cases"%len(cases))
    return cases

In [59]:
for h in s12.find_all('h2'):
        strings = [s for s in h.strings]
        if strings[0] in MONTHS.keys():
            month = str(strings[0])
            if h.next_sibling.next_sibling.name == 'ul':
                #cases = []
                for child in h.next_sibling.next_sibling.children:
                    print(child.contents)
                    print([a for a in child.strings][0])

[<i><a href="/wiki/Wikipedia:Arbitration/Requests/Case/F%C3%A6#Final_decision" title="Wikipedia:Arbitration/Requests/Case/Fæ">Wikipedia:Arbitration/Requests/Case/Fæ#Final decision</a> closed - <b>Decided</b> 21 July 2012 (<a class="external text" href="http://en.wikipedia.org/w/index.php?title=Wikipedia:Administrators%27_noticeboard&amp;diff=prev&amp;oldid=503501553">AN notice</a>) </i>]
Wikipedia:Arbitration/Requests/Case/Fæ#Final decision
[<i><a href="/wiki/Wikipedia:Arbitration/Requests/Case/GoodDay#Final_decision" title="Wikipedia:Arbitration/Requests/Case/GoodDay">Wikipedia:Arbitration/Requests/Case/GoodDay#Final decision</a> - <b>Decided</b> 14 June 2012 (<a class="external text" href="http://en.wikipedia.org/w/index.php?title=Wikipedia%3AAdministrators%27_noticeboard&amp;diff=497518703&amp;oldid=497516985">AN notice</a>)</i>]
Wikipedia:Arbitration/Requests/Case/GoodDay#Final decision
[<i><a href="/wiki/Wikipedia:Arbitration/Requests/Case/Rich_Farmbrough#Final_decision" title="Wi

In [63]:
for h in s4.find_all('h2'):
        strings = [s for s in h.strings]
        if strings[0] in MONTHS.keys():
            month = str(strings[0])
            if h.next_sibling.next_sibling.name == 'ul':
                #cases = []
                for child in h.next_sibling.next_sibling.children:
                    print(type(child))
                    print(child.contents)
                    print([a for a in child.strings][0])

<class 'bs4.element.Tag'>
['Six cases']
Six cases
<class 'bs4.element.NavigableString'>


AttributeError: 'NavigableString' object has no attribute 'contents'

In [90]:
allCases[15].get_full_url()

'https://en/wikipedia.org/wiki/Wikipedia:Requests_for_arbitration/Orthogonal'

In [94]:
'/'.join([case.url for case in allCases][0].split('/')[1:])

'wiki/Wikipedia:Requests_for_arbitration/VeryVerily'

In [84]:
outf = open('caseurls.txt', 'w')
for case in allCases:
    title = '/'.join(case.url.split('/')[2:])
    outf.write("%s\r" % title)
outf.close()

In [91]:
len(allCases)

341

In [98]:
def get_talk(url):
    base_url = '/'.join(url.split('/')[2:])
    return "%s_talk%s" % (base_url[:9], base_url[9:])

In [97]:
['/'.join(case.url.split('/')[2:])[:9] for case in allCases]

['Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wikipedia',
 'Wiki

In [100]:
tpages = [get_talk(case.url) for case in allCases]

In [101]:
import pickle
outf = open('testy/arbtalk/casetalkurls.data', 'wb')
pickle.dump(tpages, outf)
outf.close()